# Analyze fitness effects of mutations versus terminal and non-terminal counts
This notebook is designed to see if the relative number of counts of mutations on terminal and non-terminal nodes scales with the fitness effects of those mutations.

Import Python modules:

In [ ]:
import altair as alt

import numpy

import pandas as pd

import yaml

_ = alt.data_transformers.disable_max_rows()

Now get variables from `snakemake`:

In [ ]:
if "snakemake" not in globals() and "snakemake" not in locals():
    # variables set manually for interactive debugging
    aamut_all_csv = "../results/aa_fitness/aamut_fitness_all.csv"
    mutcounts_csv = "../results/mutation_counts/aggregated.csv"
    chartfile = "../results/fitness_vs_terminal/fitness_vs_terminal.html"
    
    with open("../config.yaml") as f:
        config = yaml.safe_load(f)
    min_expected_count = config["min_expected_count"]
    
else:
    # get variables from `snakemake` when running pipeline
    aamut_all_csv = snakemake.input.aamut_all
    mutcounts_csv = snakemake.input.mutcounts
    chartfile = snakemake.output.chart
    min_expected_count = snakemake.params.min_expected_count